In [126]:
#!pip install yfinance
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.optimize import minimize, Bounds, LinearConstraint
import math
import pandas as pd
import numpy as np
from scipy.stats import norm

In [133]:
start_date = "2017-01-03"
end_date = "2022-07-29"
tickerlist = ['RY.TO','TD.TO','BNS.TO','CM.TO','CWB.TO','HCG.TO','LB.TO','NA.TO','BMO.TO']

stockprice = [96.53, 64.05, 77.20, 50.06, 25.49, 25.83, 41.55, 89.49, 98.64]
initial_level = 80.28
t = 3
rfr = .03
dividendyield = .0446
vol = [0.1897, 0.2123, 0.2082, 0.2160, 0.2857, 0.6563, 0.2609, 0.2372, 0.2334]
sims = 1000
floor = 0.075
participation = 1
ceiling = 0.20
principal = 1000

weights = [0.2948,0.2498,0.1556,0.097,0.0039,0.0018,0.0030,0.0503,0.1437]

In [134]:
tickerstr = ''.join(tickerlist)
data = yf.download(tickers=tickerlist, start=start_date, end=end_date, interval="1d", group_by='ticker')

df=pd.DataFrame(index=data.index)
for x in tickerlist:
    df = pd.merge(df, data[x]['Adj Close'].rename(x), left_index=True, right_index=True)

df=df.dropna(axis=0)

df_pctchg = df.pct_change()

corr_matrix = df_pctchg.corr()

corr_matrix = np.linalg.cholesky(corr_matrix)

[*********************100%***********************]  9 of 9 completed


In [135]:
randshock = []

randshock = [ [ None for y in range(len(tickerlist)) ]
             for x in range(sims) ]

for counter1 in range (sims):
    for counter2 in range (len(tickerlist)):
        randshock[counter1][counter2] = 0
        for counter3 in range (counter2):
            randshock[counter1][counter2] = randshock[counter1][counter2] + vol[counter2]*norm.ppf(np.random.uniform(low=0,high=1))*corr_matrix[counter2][counter3]*math.sqrt(t)

In [136]:
temp_price = []

temp_price = [ [ None for y in range(len(tickerlist)) ]
             for x in range(sims) ]

dffinal = pd.DataFrame(columns=['trial','randomshock','stock','antithetic','return','antitheticreturn','effectivereturn','antieffectivereturn'])

for counter1 in range (sims):
    stockp = 0
    antistockp = 0
    for counter2 in range (len(tickerlist)):
        temp_price[counter1][counter2] = stockprice[counter2]*math.exp(((rfr-dividendyield-(vol[counter2]**2)/2)*t)+randshock[counter1][counter2])
        stockp = stockp + temp_price[counter1][counter2]*weights[counter2]
        temp_price[counter1][counter2] = stockprice[counter2]*math.exp(((rfr-dividendyield-(vol[counter2]**2)/2)*t)-randshock[counter1][counter2])
        antistockp = antistockp + temp_price[counter1][counter2]*weights[counter2]
    
    return1 = np.log(stockp/initial_level)
    antitheticreturn = np.log(antistockp/initial_level)
    dffinal = dffinal.append({'trial':counter1+1,
                    'randomshock':randshock,
                    'stock':stockp,
                    'antithetic':antistockp,
                    'return': return1,
                    'antitheticreturn': antitheticreturn,
                    'effectivereturn': max(min(return1*participation,ceiling),floor),
                    'antieffectivereturn':max(min(antitheticreturn*participation,ceiling),floor) },ignore_index=True)
  

In [137]:
dffinal['pvpayoff']= principal*(1+dffinal['effectivereturn'])*math.exp(-rfr*t)
dffinal['pvantitheticpayoff']= principal*(1+dffinal['antieffectivereturn'])*math.exp(-rfr*t)
dffinal['Averagepayoff']= (np.mean(dffinal['pvpayoff']) + np.mean(dffinal['pvantitheticpayoff']))/2

In [138]:
dffinal

,trial,randomshock,stock,antithetic,return,antitheticreturn,effectivereturn,antieffectivereturn,pvpayoff,pvantitheticpayoff,Averagepayoff
0,1,"[[0, 0.7330329484788574, -0.27082625368099505,...",96.030387,60.832226,0.179144,-0.277401,0.179144,0.075,1077.656609,982.476024,986.083127
1,2,"[[0, 0.7330329484788574, -0.27082625368099505,...",73.734233,73.536914,-0.085053,-0.087733,0.075000,0.075,982.476024,982.476024,986.083127
2,3,"[[0, 0.7330329484788574, -0.27082625368099505,...",71.293716,74.486837,-0.118712,-0.074898,0.075000,0.075,982.476024,982.476024,986.083127
3,4,"[[0, 0.7330329484788574, -0.27082625368099505,...",65.229692,81.579536,-0.207606,0.016058,0.075000,0.075,982.476024,982.476024,986.083127
4,5,"[[0, 0.7330329484788574, -0.27082625368099505,...",78.185978,67.555388,-0.026430,-0.172573,0.075000,0.075,982.476024,982.476024,986.083127
...,...,...,...,...,...,...,...,...,...,...,...
995,996,"[[0, 0.7330329484788574, -0.27082625368099505,...",82.816403,67.110835,0.031106,-0.179175,0.075000,0.075,982.476024,982.476024,986.083127
996,997,"[[0, 0.7330329484788574, -0.27082625368099505,...",76.187766,70.108003,-0.052320,-0.135484,0.075000,0.075,982.476024,982.476024,986.083127
997,998,"[[0, 0.7330329484788574, -0.27082625368099505,...",73.169147,72.471094,-0.092747,-0.102333,0.075000,0.075,982.476024,982.476024,986.083127
998,999,"[[0, 0.7330329484788574, -0.27082625368099505,...",67.049267,83.874165,-0.180093,0.043797,0.075000,0.075,982.476024,982.476024,986.083127
